# LoopTree Tutorial

LoopTree is a model to evaluate the latency and energy of a fused-layer dataflow accelerator.

To model energy and latency, a workload, architecture, and mapping have to be specified. First, we discuss how these are specified. Then, we show how to run the LoopTree model.

## Specifying Architecture, Workload, and Mapping

For the LoopTree model to estimate energy and latency, the user must specify an architecture, workload, and mapping. Below, we discuss how to specify each of these inputs.

### Architecture
In LoopTree, the architecture of an accelerator is abstracted as buffers that use the Buffet semantics and computation units, following the [Timeloop v4 specification](https://timeloop.csail.mit.edu/v4).

An example architecture that we will use here is displayed below.

In [10]:
from util import *
from pprint import pp
show_config('architecture.yaml')

variables:
  global_cycle_seconds: 1e-9
  technology: "45nm"

architecture:
  version: 0.4
  nodes:
  - !Component
    name: MainMemory
    class: DRAM
    attributes: {width: 256, block_size: 32, word_bits: 8, datawidth: 8}
    required_actions: ['read', 'write']
  - !Component
    name: GlobalBuffer
    class: SRAM
    attributes:
        depth: 8192
        width: 256
        block_size: 32
        word_bits: 8
        datawidth: 8
        n_rdwr_ports: 2
        n_rd_ports: 0
        n_wr_ports: 0
    required_actions: ['read', 'write']
  - !Component
    name: MACC
    class: intmac
    attributes:
        datawidth: 8
        width: 8
        cycle_time: 1e-9
    required_actions: ['compute']


### Workloads

DNN workloads in LoopTree are abstracted as a set of Einsums, each representing a layer.

Each Einsum is specified as a dictionary. Each Einsum reads/writes to a number of tensors (referred to as `data_spaces` in the specification), and each tensor is either an input to the operation or an output (indicated by a key "read_write" with value `True`). Moreover, a "projection" from the Einsum to each tensor, which describes the index of the element in the data that an operation in the Einsum accesses, must be specified. Finally, we specify must specify the *shape* of the Einsum, which is the bounds of its dimensions (also referred to as "ranks").

An example comprising two fully-connected layers are shown below.

In [11]:
show_config('two_fc.workload.yaml')

problem:
  - shape:
      name: Fc1
      dimensions: [ P1, M1, C1 ]
      data_spaces:
      - name: Fmap1
        dimensions: [ Fmap1_C, Fmap1_P ]
        projection: '[ C1, P1 ]'
      - name: Filter1
        dimensions: [ Filter1_C, Filter1_M ]
        projection: '[ C1, M1 ]'
      - name: Fmap2
        dimensions: [ Fmap2_C, Fmap2_P ]
        projection: '[ M1, P1 ]'
        read_write: True

    instance: >-
      0 <= P1 < 128 and 0 <= M1 < 64 and 0 <= C1 < 64

  - shape:
      name: Fc2
      dimensions: [ P2, M2, C2 ]
      data_spaces:
      - name: Fmap2
        dimensions: [ Fmap2_C, Fmap2_P ]
        projection: '[ C2, P2 ]'
      - name: Filter2
        dimensions: [ Filter2_C, Filter2_M ]
        projection: '[ C2, M2 ]'
      - name: Fmap3
        dimensions: [ Fmap3_C, Fmap3_P ]
        projection: '[ M2, P2 ]'
        read_write: True

    instance: >-
      0 <= P2 < 128 and 0 <= M2 < 64 and 0 <= C2 < 64



In this workload specification, we specify two Einsums Fc1 and Fc2. Each Einsum has three dimensions. Fc1 has dimensions P1, M1, and C1; Fc2 has dimensions P2, M2, and C2. Finally, we specify the shape of the Einsums as bounds for each Einsum dimension (rank).

### Mapping

The LoopTree mapping is a tree-structure that contains nodes of the types described below.
- **Loops**: a loop node specifies a rank in the Einsum that is partitioned and the shape of the tiles that results from the partitioning. If the loop is a "temporal" loop, then the tiles are scheduled to be processed one at a time. If the loop is a "spatial" loop, then the tiles are scheduled to parallel hardware units.
- **Branching point**: nodes above a branching point (*i.e.*, ancestor nodes) describe inter-Einsum mapping, which is applied to all Einsums under that branching point. Nodes underneath the branching point (*i.e.*, within each branch) pertain only to the Einsum of that branch.
- **Storage**: a storage node specifies the tiles of tensors to retain and which hardware level retains the tiles.
- **Compute**: a compute node specifies the hardware level used to compute operations of an Einsum. This node has to be a leaf node, and it also denotes the Einsum that a particular branch pertains to.

First, we discuss a layer-by-layer mapping example.

In [12]:
show_config('layer-by-layer.mapping.yaml')

mapping:
  type: fused
  nodes:
  - type: storage  #---------------------------------- node 1.a
    target: 0  # level 2 is bound to MainMemory
    dspace: [Filter1, Filter2, Fmap1, Fmap2, Fmap3]
  - type: sequential  #------------------------------- node 1.b
    branches:
    - - type: storage #------------------------------- node 1.c
        target: 1  # level 1 is bound to GlobalBuffer
        dspace: [Filter1]
      - type: temporal #------------------------------ node 1.d
        rank: P1
        tile_shape: 1
      - type: storage #------------------------------- node 1.e
        target: 1
        dspace: [Fmap1, Fmap2]
      - type: temporal
        rank: C1
        tile_shape: 1
      - type: temporal
        rank: M1
        tile_shape: 1
      - type: compute #------------------------------- node 1.f
        einsum: Fc1
        target: 2  # level 0 is bound to MACC
    - - type: storage #------------------------------- node 1.g
        target: 1
        dspace: [Filter2]
    

Explanation:
- (1.a) The storage node specifies that the tensors specified by the `dspace` key will be retained in target 0, which we will bind to MainMemory.
        Note that MainMemory retains Fmap2.
- (1.b) The sequential node specifies that the following branches (one for Fc1 and one for Fc2, as we will see shortly) are processed sequentially.
- (1.c) Filter1 is retained in GlobalBuffer
- (1.d) The rank P1 (which is a rank of the Fc1 Einsum) is partitioned into tiles with shape 1 and we will iterate over the tiles one at a time (temporal iteration).
- (1.e) Tiles of Fmap1 and Fmap2 are retained in GlobalBuffer. We retain tiles becuase the P1 rank has been partitioned.
- (1.f) Operations of Fc1 are processed in the MACC unit. Moreover, this node specifies that this branch is relevant to Fc2.
- (1.g) This and the following nodes specify how Fc2 is mapped.

In [13]:
show_config('fused.mapping.yaml')

mapping:
  type: fused
  nodes:
  - type: storage
    target: 0  # level 2 is bound to MainMemory
    dspace: [Filter1, Filter2, Fmap1, Fmap3] #-------- node 2.a
  - type: storage
    target: 1  # level 1 is bound to GlobalBuffer
    dspace: [Filter1, Filter2]
  - type: temporal #---------------------------------- node 2.b
    rank: P2
    tile_shape: 1
  - type: storage  #---------------------------------- node 2.c
    target: 1  # level 1 is bound to GlobalBuffer
    dspace: [Fmap1, Fmap2, Fmap3]
  - type: sequential  #------------------------------- node 2.d
    branches:
    - - type: temporal
        rank: C1
        tile_shape: 1
      - type: temporal
        rank: M1
        tile_shape: 1
      - type: compute
        einsum: Fc1
        target: 2  # level 0 is bound to MACC
    - - type: temporal
        rank: C2
        tile_shape: 1
      - type: temporal
        rank: M2
        tile_shape: 1
      - type: compute
        einsum: Fc2
        target: 2  # level 0 is bound to

Explanation:
- (2.a) Similar to (1.a), but note that Fmap2 is no longer retained in MainMemory.
- (2.b) In this mapping, the $P2$ rank is partitioned to create tiles that are rows of feature maps Fmap1, Fmap2, and Fmap3. The tile shape attribute of the loop node implies that the tiles of Fmap3 have shape 1 in the $P2$ rank. LoopTree infers the shape of the tiles of Fmap2 and Fmap1 based on what is required as inputs to compute the specified tile of Fmap3. In this case, tiles that each contains one row of Fmap2 is required to compute tiles of Fmap3, and tiles that each contains one row of Fmap1 is required to compute tiles of Fmap2 in turn.
- (2.c) The tiles of Fmap1, Fmap2, and Fmap3 are retained in GlobalBuffer.
- (2.d) Similar to before, Fc1 and Fc2 are processed sequentially. However, as we have put node (2.b) above this sequential node, *tiles* of Fc1 and Fc2 are processed sequentially in alternating fashion: a tile of Fc1 produces a tile of Fmap2, the tile of Fmap2 is consumed by a tile of Fc2, another tile of Fc1 is produced, and so on.

## Running the Model

Using the PyTimeloop library, running the model is as simple as follows.

In [14]:
from pytimeloop.looptree.run import run_looptree

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'GlobalBuffer',
    2: 'MACC'
}

latency, energy = run_looptree(
    CONFIG_DIR,
    ['architecture.yaml', 'two_fc.workload.yaml', 'layer-by-layer.mapping.yaml'],
    TMP_DIR,
    bindings,
    call_accelergy=True
)

The result returned by the model contains several information, such as latency:

In [15]:
print('Latency:', latency)

Latency: 1048576


and also energy, which is computed by calculating the number of actions to each hardware component and multiplying that with the energy per action estimated using the Accelergy tool:

In [16]:
print('Energy:')
pp(energy)

Energy:
{('MainMemory', 'read'): 83886080.0,
 ('GlobalBuffer', 'read'): 516016570.36800003,
 ('MainMemory', 'write'): 33554432.0,
 ('GlobalBuffer', 'write'): 164768251.904,
 ('MACC', 'compute'): 886046.72}


Now, we compare these results with the fused mapping:

In [17]:
from pytimeloop.looptree.run import run_looptree

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'GlobalBuffer',
    2: 'MACC'
}

latency, energy = run_looptree(
    CONFIG_DIR,
    ['architecture.yaml', 'two_fc.workload.yaml', 'fused.mapping.yaml'],
    TMP_DIR,
    bindings,
    call_accelergy=True
)
print('Latency:', latency)
print('Energy:')
pp(energy)

Latency: 1048576
Energy:
{('MainMemory', 'read'): 50331648.0,
 ('GlobalBuffer', 'read'): 516016570.36800003,
 ('MainMemory', 'write'): 16777216.0,
 ('GlobalBuffer', 'write'): 164768251.904,
 ('MACC', 'compute'): 886046.72}


As we can see, the energy consumption due to MainMemory reads and writes have decreased significantly from fusion.

Here, we have not modeled the impact of limited MainMemory bandwidth; thus, because the MACC unit utilization is the same (100%) in both cases, the latency is the same. However, if the layer-by-layer latency is limited by MainMemory bandwidth, fusion will also decrease latency.